In [ ]:
from source_files import *
from plot_helpers import *
from raster_compare.plots import PlotBase
from source_files_extended import aso_snow_depth, load_aso_depth

import query_helpers

import math

# Analysis

In [ ]:
df = db_query_to_df()

positive_sfm = query_helpers.get_positive(df, 'sfm_snow_depth')

#### Treat water and vegetation as one as likely misclassified

In [ ]:
df.loc[df['casi_class'] == 'Water', 'casi_class'] = 'Vegetation'

In [ ]:
vegetation = (positive_sfm.casi_class == 'Vegetation')
open_areas = ~vegetation

## Snow depth by elevation (positive SfM values)

In [ ]:
max_sd = math.ceil(positive_sfm.sfm_snow_depth.max())
bins = np.concatenate((
    np.arange(0, 25.10, 0.10),
    np.arange(25, 35, 0.25),
    [max_sd]
))

elevation_bands = 10
elevation_min = positive_sfm[positive_sfm['elevation'] > 0].elevation.min()
elevation_min = elevation_min - elevation_min % elevation_bands
elevation_max = positive_sfm.elevation.max() + (elevation_bands - positive_sfm.elevation.max() % elevation_bands)
elevation_range = np.arange(elevation_min, elevation_max + elevation_bands, elevation_bands)

hist_opts = dict(
    bins=[bins, elevation_range],
    vmin=0,
    vmax=5000,
    cmin=1,
)

figure_size = dict(figsize=(8, 2.75))
COLOR_BAR_ATTR = dict(right=0.9, rect=[0.91, 0.125, 0.03, 0.795])
HIST2D_PLOT = dict(ncols=2, sharey=True, **figure_size)


def plot_hist2d(ax, x_data, y_data, **kwargs):
    if 'label' in kwargs:
        ax.set_xlabel(kwargs['label'] + ' ' + SNOW_DEPTH_LABEL)
    if 'title' in kwargs:
        ax.set_title(kwargs['title'])
    ax.tick_params(axis='x', direction='inout', length=10)
    ax.tick_params(axis='y', length=6)
    ax.set_facecolor('lightgrey')
    ax.xaxis.set_ticks_position('both')
    
    data = ax.hist2d(x_data, y_data, cmap=HIST_2D_CMAP, **kwargs)
    
    ax.set_xlim(0, 40)

    return data

In [ ]:
annotate_args = dict(fontsize=14)

fig, (ax1, ax2) = plt.subplots(
    ncols=2, 
    **figure_size,
    gridspec_kw = { 'wspace': 0.45 }
)

h2d_data = plot_hist2d(
    ax1,
    positive_sfm['sfm_snow_depth'],
    positive_sfm['aso_snow_depth'],
    bins=[bins, bins],
    vmin=hist_opts['vmin'],
    vmax=hist_opts['vmax'],
    cmin=hist_opts['cmin'],
)
ax1.add_line(mlines.Line2D([-1, 16], [-1, 16], linestyle='--', color='steelblue'))
ticks = np.concatenate((np.arange(0, 11, 1), np.arange(10, 17, 2)))
ax1.set_xticks(ticks)
ax1.set_xlim(0, 10)
ax1.set_xlabel('SfM ' + SNOW_DEPTH_LABEL)
ax1.set_yticks(ticks)
ax1.set_ylim(0, 10)
ax1.set_ylabel('ASO ' + SNOW_DEPTH_LABEL, labelpad=0)
ax1.xaxis.set_ticks_position('bottom')
ax1.annotate('a)', (0.25, 9.1), **annotate_args)

color_bar = PlotBase.insert_colorbar(ax1, h2d_data[3], '')
color_bar.set_label(label='Count', rotation=270, labelpad=7.5)

Histogram.add_to_plot(
    ax2, 
    data=np.ma.asarray(positive_sfm['aso_snow_depth']), 
    label='ASO', 
    color='steelblue',
    skip_mean=True
)
Histogram.add_to_plot(
    ax2, 
    data=np.ma.asarray(positive_sfm['sfm_snow_depth']), 
    label='SfM', 
    color='beige', 
    alpha=0.7,
    skip_mean=True
)


ax2.set_xlim(0, 5)
ax2.set_xlabel(SNOW_DEPTH_LABEL)

PlotBase.format_axes_scientific(
    ax2, 'y', (4, 4), labelpad=0, rotation=90, fontsize=LABEL_SIZE - 1
)

ax2.legend(loc='upper right', )
handles, labels = ax2.get_legend_handles_labels()

ax2.legend(
    handles=handles, 
    fontsize=LABEL_SIZE - 1, labelspacing=0.3, borderaxespad=0.2,
    frameon=False, 
)
ax2.annotate('b)', (0.25, 155000), **annotate_args);

In [ ]:
annotate_args = dict(xy=(16.5, 3950), fontsize=14)
ticks = np.arange(0, 20, step=2)

fig, (ax1, ax2, cax) = plt.subplots(
    ncols=3, 
    figsize=(8, 2.6),
    gridspec_kw = { 
        'wspace': 0.1, 
        'width_ratios': [.485, .485, .03]
    }
)

plot_hist2d(
    ax1,
    positive_sfm['sfm_snow_depth'],
    positive_sfm['elevation'],
    label='SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL, labelpad=4)
ax1.annotate('a)', **annotate_args)
ax1.set_xlim(ticks.min(), ticks.max())
ax1.set_xticks(ticks)

h2d_data_aso = plot_hist2d(
    ax2,
    positive_sfm['aso_snow_depth'],
    positive_sfm['elevation'],
    label='ASO',
    **hist_opts,
)

ax2.set_xlim(ticks.min(), ticks.max())
ax2.set_xticks(ticks)
ax2.set_yticklabels([])
ax2.annotate('b)', **annotate_args)

color_bar = fig.colorbar(h2d_data_aso[3], cax=cax)
color_bar.set_label(label='Count', rotation=270, labelpad=7.5)

In [ ]:
annotate_args = dict(xy=(18, 3980), fontsize=16)

fig = plt.figure(figsize=(8, 6))
gs_top = fig.add_gridspec(
    ncols=3, nrows=1, wspace=0.08, 
    width_ratios=[.485, .485, .03], 
    bottom=0, top=.46
)
gs_bottom = fig.add_gridspec(
    ncols=2, nrows=1, bottom=0.54, top=1
)

ax1 = fig.add_subplot(gs_top[0, 0])
plot_hist2d(
    ax1,
    positive_sfm['sfm_snow_depth'],
    positive_sfm['elevation'],
    label='SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL)
ax1.annotate('c)', **annotate_args)
ax1.set_xlim(0, 20)

ax2 = fig.add_subplot(gs_top[0, 1])
h2d_data_aso = plot_hist2d(
    ax2,
    positive_sfm['aso_snow_depth'],
    positive_sfm['elevation'],
    label='ASO',
    **hist_opts,
)

ax2.set_xlim(0, 20)
ax2.set_yticklabels([])
ax2.annotate('d)', **annotate_args)

color_bar = fig.colorbar(h2d_data_aso[3], cax=fig.add_subplot(gs_top[0, 2]))
color_bar.set_label(label='Count', rotation=270, labelpad=10)

ax3 = fig.add_subplot(gs_bottom[0, 0])
h1 = plot_hist2d(
    ax3,
    positive_sfm['sfm_snow_depth'],
    positive_sfm['aso_snow_depth'],
    bins=[bins, bins],
    vmin=0,
    vmax=150,
    cmin=1,
)
ax3.add_line(mlines.Line2D([-1, 16], [-1, 16], linestyle='--', color='steelblue'))
ticks = np.concatenate((np.arange(0, 11, 1), np.arange(10, 17, 2)))
ax3.set_xticks(ticks)
ax3.set_xlim(0, 15)
ax3.set_xlabel('SfM ' + SNOW_DEPTH_LABEL)
ax3.set_yticks(ticks)
ax3.set_ylim(0, 15)
ax3.set_ylabel('ASO ' + SNOW_DEPTH_LABEL)
ax3.xaxis.set_ticks_position('bottom')
ax3.annotate('a)', (0.5, 13.6), fontsize=16)

ax4 = fig.add_subplot(gs_bottom[0, 1])
Histogram.add_to_plot(
    ax4, 
    data=np.ma.asarray(positive_sfm['aso_snow_depth']), 
    label='ASO', 
    color='steelblue'
)
Histogram.add_to_plot(
    ax4, 
    data=np.ma.asarray(positive_sfm['sfm_snow_depth']), 
    label='SfM', 
    color='beige', 
    alpha=0.7
)
PRECISION_LINE = dict(linestyle='dotted', color='dimgrey')
ax4.axvline(x=0.08, **PRECISION_LINE)
ax4.set_xlim(0, 5)
ax4.set_xlabel(SNOW_DEPTH_LABEL)

PlotBase.move_yaxis_label_right(ax4)
PlotBase.format_axes_scientific(ax4, 'y', (4, 4))
ax4.legend(loc='upper right', )
handles, labels = ax4.get_legend_handles_labels()
handles.append(
    mlines.Line2D(
        [], [], 
        label='Means', 
        linestyle=Histogram.MEAN_LINE_STYLE['linestyle'],
        color=Histogram.MEAN_LINE_STYLE['color'],
    )
)
handles.append(
    mlines.Line2D(
        [], [], label='ASO Uncertainty', **PRECISION_LINE
    )
)
ax4.legend(
    handles=handles, fontsize='large', frameon=False, labelspacing=0.3, borderaxespad=0.2
)
ax4.annotate('b)', (4.5, 157000), fontsize=16);

In [ ]:
annotate_args = dict(xy=(27, 3980), fontsize=16)

fig = plt.subplots(**figure_size)
gs1 = GridSpec(1, 2)
gs1.update(left=0., right=0.63, wspace=0.05)

ax1 = plt.subplot(gs1[0, 0])
h2d_data_sfm = plot_hist2d(
    ax1,
    positive_sfm['sfm_snow_depth'],
    positive_sfm['elevation'],
    label='SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL)
ax1.annotate('a)', **annotate_args)
ax1.set_xlim(0, 30)

ax2 = plt.subplot(gs1[0, 1])
h2d_data_aso = plot_hist2d(
    ax2,
    positive_sfm['aso_snow_depth'],
    positive_sfm['elevation'],
    label='ASO',
    **hist_opts,
)
ax2.set_xlim(0, 30)
ax2.set_yticklabels([])
ax2.annotate('b)', **annotate_args)

gs2 = GridSpec(1, 1)
gs2.update(left=0.70, right=1)
ax3 = plt.subplot(gs2[0, 0])

h1 = plot_hist2d(
    ax3,
    positive_sfm['sfm_snow_depth'],
    positive_sfm['aso_snow_depth'],
    bins=[bins, bins],
    vmin=0,
    vmax=250,
    cmin=1,
)
ax3.add_line(mlines.Line2D([-1, 16], [-1, 16], linestyle='--', color='steelblue'))
ticks = np.concatenate((np.arange(0, 11, 1), np.arange(10, 17, 2)))
ax3.set_xticks(ticks)
ax3.set_xlim(0, 15)
ax3.set_xlabel('SfM ' + SNOW_DEPTH_LABEL)
ax3.set_yticks(ticks)
ax3.set_ylim(0, 15)
ax3.set_ylabel('ASO ' + SNOW_DEPTH_LABEL)
ax3.xaxis.set_ticks_position('bottom')
ax3.annotate('c)', (0.5, 13.6), fontsize=16)
PlotBase.insert_colorbar(ax3, h1[3], 'Count');

In [ ]:
fig, (ax1, ax2) = plt.subplots(sharex=True, **HIST2D_PLOT)

bin_count_sum = np.count_nonzero(~np.isnan(h2d_data_sfm[0].T), axis=1)
ax1.axhline(y=3550, color='orange', linestyle='--')
ax1.plot(bin_count_sum, elevation_range[:-1])#, s=10)
ax1.set_ylabel(ELEVATION_LABEL)
ax1.set_xlabel('Number of bins')

bin_count_sum = np.count_nonzero(~np.isnan(h2d_data_aso[0].T), axis=1)
ax2.axhline(y=3550, color='orange', linestyle='--')
ax2.plot(bin_count_sum, elevation_range[:-1])#, s=10)
ax2.set_xlabel('Number of bins')

plt.suptitle('Bin count per elevation band', fontsize=16)
ax1.set_xlim(left=0);

In [ ]:
fig, (ax1, ax2) = plt.subplots(**HIST2D_PLOT)

h1 = plot_hist2d(
    ax1,
    positive_sfm[vegetation]['sfm_snow_depth'],
    positive_sfm[vegetation]['elevation'],
    label='SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL)

h2 = plot_hist2d(
    ax2,
    positive_sfm[vegetation]['aso_snow_depth'],
    positive_sfm[vegetation]['elevation'],
    label='ASO',
    **hist_opts,
)

plt.suptitle('Snow Depth in Vegetation')
PlotBase.insert_colorbar(ax2, h2[3], 'count', **COLOR_BAR_ATTR);

In [ ]:
fig, (ax1, ax2) = plt.subplots(**HIST2D_PLOT)

h1 = plot_hist2d(
    ax1,
    positive_sfm[open_areas]['sfm_snow_depth'],
    positive_sfm[open_areas]['elevation'],
    label='SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL)


h2 = plot_hist2d(
    ax2,
    positive_sfm[open_areas]['aso_snow_depth'],
    positive_sfm[open_areas]['elevation'],
    label='ASO',
    **hist_opts,
)
plt.suptitle('Snow Depth in Open Areas')
PlotBase.insert_colorbar(ax2, h2[3], 'count', **COLOR_BAR_ATTR);

# Snow Depth Difference (SfM - ASO)

In [ ]:
cmap = plt.get_cmap("tab20c")
cmap = cmap(np.arange(4)*4, alpha=0.55)
cmap = [cmap[1], cmap[0], cmap[2]]

In [ ]:
casi_classes, classes_count = np.unique(df.casi_class.to_numpy(), return_counts=True)    

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, **figure_size)
bin_width = 0.1
bins = np.arange(
    math.floor(df.sd_difference.min()) - bin_width, 
    math.ceil(df.sd_difference.max()) + bin_width, 
    bin_width
)
stack = []

for casi_class in casi_classes:
    stack.append(df[df.casi_class == casi_class].sd_difference)
    
ax1.hist(
    stack,
    bins=bins,
    label=casi_classes,
    stacked=True,
    color=cmap,
)
ax1.axvline(0, color='black', linewidth=.4, alpha=1)
ax1.set_xlabel(SNOW_DEPTH_LABEL)
ax1.set_xlim(-5, 5)
ax1.set_ylabel('Count')
ax1.legend(loc='upper right')

patches, texts, autotexts = ax2.pie(
    classes_count, 
    labels=casi_classes, 
    colors=cmap,
    wedgeprops=dict(width=0.5, edgecolor='w'),
    textprops={'fontsize': 14},
    autopct='%1.1f%%')
ax2.axis('equal');